Imports

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
from nltk.corpus import stopwords
from nltk import re
from nltk import word_tokenize
from nltk import tokenize
import unicodedata
import gensim

In [ ]:
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os

Funciones utiles

In [ ]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    fp = open(path, 'rb')
    iterPDF = iter(PDFPage.get_pages(fp, set(), maxpages=0, password="",caching=True, check_extractable=True))
    next(iterPDF)

    for page in iterPDF:
        interpreter.process_page(page)
        text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [ ]:
def preprocess_text(text):
    text = text.lower() 
    
    text = text.replace('.','PUNTOSEGUIDO')
    #eliminar signos de puntuacion
    #words = [word for word in word_tokenize(text) if re.search("\w", word)]
    
    #eliminar numeros
    text = ''.join([word for word in text if not word.isdigit()])
    
    #eliminar caracteres unicode
    text = unicodedata.normalize("NFKD", text).encode("ascii","ignore").decode("ascii")
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', text)
    
    #reemplazar espacios consecutivos por uno solo
    #text = re.compile(r"\s+").sub(" ", text).strip()
    
    #eliminar algunas stop words particulares de los documentos
    special_stop_words = ['cuestionario','preguntas','respuestas','responder','trabajo','practico','marketing','digital','cuatrimestre','pagina']
    text  = ' '.join([word.lower() for word in re.split("\W+",text) if word.lower() not in special_stop_words])
    
    #eliminar palabras de menos de 1 letra que no sea vocal
    text  = ' '.join([word.lower() for word in re.split("\W+",text) if len(word)>1 or word in ['a','e','i','o','u']])
    
    text = text.replace('puntoseguido','.')
    return text

In [ ]:
def is_in_vocabulary(sentence):
    words = tokenize.word_tokenize(sentence)
    return [word for word in words if word in trained_model.wv.vocab ]

In [ ]:
def is_a_question_word(word):
    return word in ['cuales','que','de','explique','desarrolle','describa','¿']

Cargar modelo entrenado

In [ ]:
def cargar_modelo(path):
    return KeyedVectors.load(path)

In [ ]:
trained_model = cargar_modelo ("modelo_entrenado_dataset_UTN.bin")

In [ ]:
trained_model = cargar_modelo ("complete.kv")

In [ ]:
def spacy_model(doc1,doc2,nlp):
    sentences1 = tokenize.sent_tokenize(doc1)
    sentences2 = tokenize.sent_tokenize(doc2)
    
    for sentence1 in sentences1:
        for sentence2 in sentences2:
            
            words1 = tokenize.word_tokenize(sentence1)
            words2 = tokenize.word_tokenize(sentence2)            
            
            if(len(words1)>5 and len(words2)>5):
                sentence = " ".join([word for word in words1 if word not in stopwords.words('spanish')])
                sentencee = " ".join([word for word in words2 if word not in stopwords.words('spanish')])
                
                similaridad = nlp(sentence).similarity(nlp(sentencee))
                
                if similaridad > 0.9 and not is_a_question_word(words1[0]) and not is_a_question_word(words2[0]):
                    print("POSIBLE PLAGIO DETECTADO \n ORACION1: " +  sentence1 + "\n ORACION2: " + sentence2 + "\n SIMILITUD: " + str(similaridad))
                    print('--------------------O-----------------')
    

In [66]:
def chequear_plagio_similitudes(doc1,doc2,file):
    sentences1 = tokenize.sent_tokenize(doc1)
    sentences2 = tokenize.sent_tokenize(doc2)
    cant_similitudes = 0
    plagios=""
    
    for sentence1 in sentences1:
        words1 = is_in_vocabulary(sentence1)
        print("hola")
        if(len(words1)>3):
            for sentence2 in sentences2:
                words2 = is_in_vocabulary(sentence2)
                if(len(words2)>3):

                    for word1 in words1:
                        cant_similitudes = 0
                        for word2 in words2:
                            if(word2 in trained_model.most_similar(word1,topn=30)):
                                cant_similitudes = cant_similitudes + 1

                if cant_similitudes >= 1 and not is_a_question_word(words1[0]) and not is_a_question_word(words2[0]):
                    print("POSIBLE PLAGIO DETECTADO \n ORACION1: " +  sentence1 + "\n ORACION2: " + sentence2 + "\n SIMILITUD: " + str(similaridad))
                    print('--------------------O-----------------')
        
    return plagios

In [ ]:
def chequear_plagio(doc1,doc2,file):
    print(doc1)
    
    sentences1 = tokenize.sent_tokenize(doc1)
    sentences2 = tokenize.sent_tokenize(doc2)
    
    plagios = ""
    
    for sentence1 in sentences1:
        for sentence2 in sentences2:
            
            similaridad = 0
            
            words1 = is_in_vocabulary(sentence1)
            words2 = is_in_vocabulary(sentence2)
            print(sentence1)
            if(len(words1)>5 and len(words2)>5):
                similaridad = trained_model.n_similarity(words1,words2)
                print(similaridad)
                if similaridad > 0.1 and not is_a_question_word(words1[0]) and not is_a_question_word(words2[0]):
                    plagios = plagios + '\n--------------------O----------------- \n TP PLAGIADO: '  + file  +  "\n ORACION1: " +  sentence1 + "\n ORACION2: " + sentence2 + "\n SIMILITUD: " + str(similaridad)
            #        print("POSIBLE PLAGIO DETECTADO \n ORACION1: " +  sentence1 + "\n ORACION2: " + sentence2 + "\n SIMILITUD: " + str(similaridad))
          #          print('--------------------O-----------------')
        
        return plagios

In [ ]:
pdf_to_check = preprocess_text(convert_pdf_to_txt("D:\LaboratoriosNLP\TP NLP\Input\SCHMID TP N°3 Experience Economy.pdf"))
directorio_data_sets = r'D:\TP NLP\DataSet'
doc = preprocess_text(convert_pdf_to_txt("D:\TP NLP\DataSet\K5071 - Matias David Choren - TP N°5 Rifkin (1).pdf"))

chequear_plagio_similitudes(doc,pdf_to_check,"Rikfin")


hola
hola


In [41]:
print(doc)

matias david choren . a .puede describir el vinculo entre las leyes de la termodinamica de newton la factura entropica .podria caracterizar la primera segunda revolucion industrial al decir de rifkin que inventos son las metaforas de cada infraestructura en cada una de esas etapas .que dice rifkin que la internet de las cosas iot le aportara a la ra revolucion industrial .que entiende rifkin por una produccion abierta distribuida colaborativa la innovacion bajo el procomun .que ejemplos actuales de procomunes se le ocurren describa. que limites le ve ud. a los procomunes como forma de produccion .a la sociedad economia del conocimiento entendida como la ra revolucion de rifkin cual de las infraestructuras claves estaria mas demorada porque que estaria faltando .como funcionaria o se garantizaria comparado con el sistema capitalista .acorde su lectura de rifkin las clases de prince para el estadio actual de la economia elija uno de estos apellidos justifique brevemente. e. colaborativa 

In [ ]:
doc = preprocess_text(convert_pdf_to_txt(directorio_data_sets + r'\TP3PabloPallocchi.pdf'))
chequear_plagio(doc,pdf_to_check,[])

In [ ]:
import es_core_news_sm
nlp = es_core_news_sm.load()
spacy_model(doc,pdf_to_check,nlp)

In [ ]:
class Plagio:
    def __init__(self,oracion1,oracion2,cosineDistance,doc1,doc2):
        self.oracion1 = oracion1
        self.oracion2 = oracion2
        self.cosineDistance = str(cosineDistance)
        self.documento1 = doc1
        self.documento2 = doc2

In [ ]:
df_plagios.to_csv("plagios.csv")
    